# Goal 3: Calibration and Rectification

### Step #01: Load images 

In [41]:
import numpy as np
import cv2
import glob
import os
import datetime

In [42]:
# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------
RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

In [43]:
# Board definitions
boards = [
    {"name": "boardBig", "pattern_size": (11,7)},
    {"name": "boardMed", "pattern_size": (5,7)},
]

# VISUALIZATION SETTINGS
VISUALIZE = True
VIS_DELAY_MS = 0  # Wait 500ms between images. Set to 0 to wait for keypress.

# !!! CRITICAL !!! 
# Measure your real square size. 
# USE METERS to match KITTI format (e.g., 30mm = 0.03)
REAL_SQUARE_SIZE_M = 0.1

# Precompute object point templates
obj_templates = {}
for b in boards:
    nx, ny = b["pattern_size"]
    # We apply the real world scale here
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * REAL_SQUARE_SIZE_M
    obj_templates[b["name"]] = objp

In [ ]:
import cv2
import numpy as np
import glob
import os

# ---------------------------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------------------------

RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

boards = [
    {"name": "boardBig", "pattern_size": (11, 7)}, # 12x8 squares
    {"name": "boardMed", "pattern_size": (5, 7)},  # 6x8 squares
]

VISUALIZE = True

# ---------------------------------------------------------------------------
# 2. DETECTION LOGIC (No Drawing)
# ---------------------------------------------------------------------------

def get_image_files(directory):
    if not os.path.exists(directory): return []
    files = sorted(glob.glob(os.path.join(directory, "*.png")))
    if not files: files = sorted(glob.glob(os.path.join(directory, "*.jpg")))
    return files

def sharpen_image(gray_img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(gray_img, -1, kernel)

def detect_boards_raw(img):
    """
    Returns a list of dicts: {'name': str, 'centroid': (x,y), 'corners': np.array}
    Does NOT draw anything on the image.
    """
    gray_base = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_processing = sharpen_image(gray_base)
    
    # Subpixel criteria
    subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 0.001)
    
    detections = []

    for board in boards:
        name = board["name"]
        nx, ny = board["pattern_size"]
        
        while True:
            # 1. Standard Detect
            flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            ret, corners = cv2.findChessboardCorners(gray_processing, (nx, ny), flags)
            
            # 2. Upscale Detect (for far objects)
            if not ret:
                gray_upscaled = cv2.resize(gray_processing, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
                ret, corners_up = cv2.findChessboardCorners(gray_upscaled, (nx, ny), flags)
                if ret:
                    corners = corners_up / 2.0

            if ret:
                # 3. Refine
                corners_refined = cv2.cornerSubPix(gray_base, corners, (11, 11), (-1, -1), subpix_criteria)
                
                # 4. Calculate Centroid (Arithmetic Mean)
                cx, cy = np.mean(corners_refined, axis=0).ravel()
                
                detections.append({
                    'name': name,
                    'centroid': (cx, cy),
                    'corners': corners_refined # Kept for masking logic
                })

                # 5. Masking (Black out found board)
                corners_int = corners.astype(np.int32)
                hull = cv2.convexHull(corners_int)
                mask = np.zeros_like(gray_processing)
                cv2.fillConvexPoly(mask, hull, 255)
                mask_dilated = cv2.dilate(mask, np.ones((15, 15), np.uint8))
                gray_processing[mask_dilated > 0] = 0
            else:
                break
                
    return detections

# ---------------------------------------------------------------------------
# 3. STEREO MATCHING LOGIC
# ---------------------------------------------------------------------------

def match_boards_by_alignment(left_dets, right_dets):
    """
    Matches boards based on Y-Coordinate Alignment (Epipolar Constraint).
    """
    matches = [] 
    used_right_indices = set()
    Y_TOLERANCE = 30.0 

    for l_det in left_dets:
        best_r_idx = -1
        best_y_diff = float('inf')
        
        lx, ly = l_det['centroid']

        for r_idx, r_det in enumerate(right_dets):
            if r_idx in used_right_indices:
                continue
            
            if l_det['name'] != r_det['name']:
                continue

            rx, ry = r_det['centroid']
            y_diff = abs(ly - ry)
            
            if y_diff < Y_TOLERANCE:
                if y_diff < best_y_diff:
                    best_y_diff = y_diff
                    best_r_idx = r_idx
        
        if best_r_idx != -1:
            matches.append((l_det, right_dets[best_r_idx]))
            used_right_indices.add(best_r_idx)
        else:
            matches.append((l_det, None))
            
    return matches

# ---------------------------------------------------------------------------
# 4. MAIN EXECUTION
# ---------------------------------------------------------------------------

def get_unique_color(seed_val):
    """Generates a unique, bright BGR color based on an integer seed."""
    np.random.seed(seed_val * 999) # Deterministic random color
    # Generate high values (100-255) for visibility on dark backgrounds
    return tuple(np.random.randint(100, 255, 3).tolist())

if __name__ == "__main__":
    left_files = get_image_files(LEFT_IMG_PATH)
    right_files = get_image_files(RIGHT_IMG_PATH)

    if not left_files:
        print("Error: Images not found.")
        exit()

    # Process First Pair
    f_left = left_files[0]
    f_right = right_files[0]
    
    print(f"Processing Pair: {os.path.basename(f_left)}")

    imgL = cv2.imread(f_left)
    imgR = cv2.imread(f_right)

    if imgL is not None and imgR is not None:
        
        # 1. Detect
        print("Detecting Left...")
        detsL = detect_boards_raw(imgL)
        print(f"Detecting Right... ({len(detsL)} found in Left)")
        detsR = detect_boards_raw(imgR)

        # 2. Match
        matched_pairs = match_boards_by_alignment(detsL, detsR)

        # 3. Visualize
        h, w, _ = imgL.shape
        canvas = np.hstack((imgL, imgR))
        
        # Darken the image so colorful centroids pop out
        canvas = (canvas * 0.3).astype(np.uint8)

        for i, (l_det, r_det) in enumerate(matched_pairs):
            
            # Generate a unique color for this match
            color = get_unique_color(i)
            
            # Left Centroid
            lx, ly = l_det['centroid']
            pt_L = (int(lx), int(ly))
            
            # Draw Left
            cv2.circle(canvas, pt_L, 8, color, -1)
            cv2.circle(canvas, pt_L, 10, (255,255,255), 1) # White outline
            
            # Label Left (L + Index)
            cv2.putText(canvas, f"L{i}", (pt_L[0]-10, pt_L[1]-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if r_det:
                # Right Centroid
                rx, ry = r_det['centroid']
                # Shift X coordinate by width of left image
                pt_R_canvas = (int(rx) + w, int(ry))
                
                # Draw Right
                cv2.circle(canvas, pt_R_canvas, 8, color, -1)
                cv2.circle(canvas, pt_R_canvas, 10, (255,255,255), 1) # White outline
                
                # Label Right (R + Index)
                cv2.putText(canvas, f"R{i}", (pt_R_canvas[0]-10, pt_R_canvas[1]-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
                # Draw Connection Line
                cv2.line(canvas, pt_L, pt_R_canvas, color, 2)
                
                # Draw Disparity Info
                disparity = lx - rx
                mid_point = (int((pt_L[0] + pt_R_canvas[0])/2), int((pt_L[1] + pt_R_canvas[1])/2))
                cv2.putText(canvas, f"dx:{disparity:.0f}", mid_point, 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
                print(f"MATCH {i}: {l_det['name']} | Disparity: {disparity:.2f}px")
            else:
                # Draw Red X for unmatched
                cv2.drawMarker(canvas, pt_L, (0, 0, 255), cv2.MARKER_TILTED_CROSS, 15, 2)
                print(f"UNMATCHED {i}: {l_det['name']} at Left ({lx:.1f}, {ly:.1f})")

        if VISUALIZE:
            scale = 0.7
            canvas_resized = cv2.resize(canvas, None, fx=scale, fy=scale)
            
            cv2.imshow('Stereo Centroid Matches', canvas_resized)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

Found 19 Left images and 19 Right images.
Starting processing loop. Press 'q' or 'ESC' on the window to stop.

--- [1/19] Processing: 0000000000.png ---
 > Detections: Left=12, Right=12
   MATCH 0: boardBig | Disparity: 68.38px
   MATCH 1: boardBig | Disparity: 67.90px
   MATCH 2: boardMed | Disparity: 95.51px
   MATCH 3: boardMed | Disparity: 71.82px
   MATCH 4: boardMed | Disparity: 93.31px
   MATCH 5: boardMed | Disparity: 110.22px
   MATCH 6: boardMed | Disparity: 128.73px
   MATCH 7: boardMed | Disparity: 91.86px
   MATCH 8: boardMed | Disparity: 84.21px
   MATCH 9: boardMed | Disparity: 79.89px
   MATCH 10: boardMed | Disparity: 97.05px
   MATCH 11: boardMed | Disparity: 123.21px

--- [2/19] Processing: 0000000001.png ---
 > Detections: Left=12, Right=12
   MATCH 0: boardBig | Disparity: 67.85px
   MATCH 1: boardBig | Disparity: 67.98px
   MATCH 2: boardMed | Disparity: 96.19px
   MATCH 3: boardMed | Disparity: 72.41px
   MATCH 4: boardMed | Disparity: 93.20px
   MATCH 5: boardMe